## Below is the code to re-create Figure 4

### Topic: Generating clusters with multiplicity and differential extinction

In [1]:
# Import necessary packages. 
from popstar import synthetic, evolution, atmospheres, reddening, ifmr
from popstar.imf import imf, multiplicity
import numpy as np
import pylab as py
import pdb
import os
import pylab as py

/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:329: UserWarning: Extinction files not found in /Users/mwhosek/models/models_2019_07/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [4]:
# First, we'll generate 2 cluster CMDs, one with multiplicity and one without. We'll 
# reuse the merged isochrone (iso_merged) with Cardelli+89 extinction law from Figure 2 
# for the purposes  of the plot (Age = 5 Myr, solar metallicity, A_Ks = 1.0 mag, dist=4000 pc).
# Both clusters will be 10^4 M_sun and a Kroupa+01 IMF

# Fetch isochrone
logAge = 6.70 # Age in log(years)
AKs = 1.0 # Ks filter extinction in mags
dist = 4000 # distance in parsecs
metallicity = 0 # metallicity in [M/H]
atm_func = atmospheres.get_merged_atmosphere
evo_merged = evolution.MergedBaraffePisaEkstromParsec()
redlaw = reddening.RedLawCardelli(3.1) # Rv = 3.1
filt_list = ['nirc2,J', 'nirc2,Kp']

iso_dir = 'iso_merged_r1/'
iso_merged = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_merged, atm_func=atm_func,
                                 filters=filt_list, red_law=redlaw,
                                 iso_dir=iso_dir, mass_sampling=3)

In [10]:
from importlib import reload
reload(imf)
reload(synthetic)

# Now we can make the cluster. 
clust_mtot = 10**4.
clust_multiplicity = multiplicity.MultiplicityUnresolved()

# Multiplicity is defined in the IMF object
massLimits = np.array([0.08, 0.5, 1, 120])
powers = np.array([-1.3, -2.3, -2.3])
#clust_imf_noMult = imf.IMF_broken_powerlaw(massLimits, powers, 
#                                           multiplicity=None)
#clust_imf_Mult = imf.IMF_broken_powerlaw(massLimits, powers, 
#                                           multiplicity=clust_multiplicity)
clust_imf_noMult = imf.Kroupa_2001(multiplicity=None)
clust_imf_Mult = imf.Kroupa_2001(multiplicity=clust_multiplicity)

In [18]:
# Make clusters
clust_noMult = synthetic.ResolvedCluster(iso_merged, clust_imf_noMult, clust_mtot)
clust_Mult = synthetic.ResolvedCluster(iso_merged, clust_imf_Mult, clust_mtot)

clust_noMult = clust_noMult.star_systems
clust_Mult = clust_Mult.star_systems

Found 8693 stars out of mass range
Found 5472 stars out of mass range
Found 986 companions out of stellar mass range


/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)
/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


In [25]:
# Next, we'll generate a cluster with multiplicity off *but* with 
# differential extinction. Note that we have to use the 
# ResolvedClusterDiffRedden object now, and we have to pass in dAKs and 
# the extinction law. 
#
# The amount of differential extinction in a system (i.e., difference between 
# the system extinction and the average extinction) is drawn from a Gaussian
# distribution centered at 0 and of width dAKs

dAKs = 0.05 # Width of differential extinction distribution in K-band, in mags
clust_noMult_diff = synthetic.ResolvedClusterDiffRedden(iso_merged, clust_imf_noMult, 
                                                        clust_mtot, dAKs, red_law=redlaw)
clust_noMult_diff = clust_noMult_diff.star_systems

/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)
/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


In [27]:
# Now we can make the final cluster figure
py.figure(figsize=(20,10))
py.subplots_adjust(left=0.08)
py.subplot(121)
py.plot(clust_Mult['m_nirc2_J'] - clust_Mult['m_nirc2_Kp'],
       clust_Mult['m_nirc2_J'], 'r.', label='Cluster w/ Multiples')
py.plot(clust_noMult['m_nirc2_J'] - clust_noMult['m_nirc2_Kp'],
       clust_noMult['m_nirc2_J'], 'k.', label='Cluster w/o Multiples')
py.xlabel('NIRC2 J - Kp (mag)', fontsize=24)
py.ylabel('NIRC2 J (mag)', fontsize=24)
py.gca().invert_yaxis()
py.tick_params(axis='both', labelsize=20)
py.legend()
py.subplot(122)
py.plot(clust_noMult_diff['m_nirc2_J'] - clust_noMult_diff['m_nirc2_Kp'],
       clust_noMult_diff['m_nirc2_J'], 'r.', label='Cluster w/ dAKs = 0.05 mag')
py.plot(clust_noMult['m_nirc2_J'] - clust_noMult['m_nirc2_Kp'],
       clust_noMult['m_nirc2_J'], 'k.', label='Cluster w/o Multiples, dAKs')
py.xlabel('NIRC2 J - Kp (mag)', fontsize=24)
py.ylabel('NIRC2 J (mag)', fontsize=24)
py.gca().invert_yaxis()
py.tick_params(axis='both', labelsize=20)
py.legend()
py.savefig('cluster_cmd.pdf', format='pdf')